<a href="https://colab.research.google.com/github/Derinhelm/parser_stat/blob/main/udpipe_spacy_getting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SynTagRus loading (test part of dataset)

In [ ]:
!wget -m https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu
!mv /content/raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu /content/ru_syntagrus-ud-test.conllu
!rm -r /content/raw.githubusercontent.com

--2024-07-31 20:57:59--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14970966 (14M) [text/plain]
Saving to: ‘raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu’

raw.githubuserconte 100%[===================>]  14.28M  --.-KB/s    in 0.1s    

Last-modified header missing -- time-stamps turned off.
2024-07-31 20:57:59 (100 MB/s) - ‘raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu’ saved [14970966/14970966]

FINISHED --2024-07-31 20:57:59--
Total wall clock time: 0.3s
Downloaded: 1 files, 14M in 0.1s (100 MB/s)


# SynTagRus data reading

In [ ]:
class ConllEntry:
    def __init__(self, id, form=None, lemma=None, pos=None, cpos=None, feats=None, \
                 parent_id=None, relation=None,deps=None, misc=None):
        self.id = id
        self.form = form
        self.lemma = lemma
        self.pos = pos
        self.cpos = cpos
        self.feats = feats
        self.parent_id = parent_id
        self.relation = relation
        self.deps = deps
        self.misc = misc

In [ ]:
class Sentence:
    def __init__(self):
        self.text = ""
        self.sent_id = ""
        self.tokens = []

    def add_token(self, token):
        self.tokens.append(token)

    def set_sent_id(self, sent_id):
        self.sent_id = sent_id

    def set_text(self, text):
        self.text = text

    def is_not_empty(self):
        return len(self.tokens) > 0

In [ ]:
def get_dataset_sentences(dataset_path):
    fh = open(dataset_path,'r',encoding='utf-8')
    sents_read = 0
    sents = []
    comments = set()

    sent = Sentence()
    for line in fh:
        tok = line.strip().split('\t')
        if not tok or line.strip() == '': # empty line, add sentence to list
            if sent.is_not_empty:
                sents_read += 1
                sents.append(sent)
            sent = Sentence()
        else:
            if line[0] == '#' or '-' in tok[0]: # a comment line
                line = line.strip()
                if line[:12] == "# sent_id = ":
                    sent.set_sent_id(line[12:])
                elif line[:9] == "# text = ":
                    sent.set_text(line[9:])
                else:
                    comments.add(line)

            else: # an actual ConllEntry, add to tokens
                if tok[2] == "_":
                    tok[2] = tok[1].lower()

                word = ConllEntry(*tok)
                sent.add_token(word)
    fh.close()
    return sents

In [ ]:
sents = get_dataset_sentences("/content/ru_syntagrus-ud-test.conllu")

In [ ]:
len(sents)

8800

# UDPipe (from spacy)

In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install spacy-udpipe==1.0.0
clear_output()

In [ ]:
import spacy_udpipe

spacy_udpipe.download("ru")

class UDPipeSpacyParser:
    def __init__(self):
        self.udpipe_model = spacy_udpipe.load("ru")

    def parse(self, sent):
        parsed = self.udpipe_model(sent)
        res = []
        for token in parsed:
            if token.dep_ != 'ROOT':
              parent_id = token.head.i+1
              relation = token.dep_
            else:
              parent_id = 0
              relation = 'root'
            c = ConllEntry(str(token.i+1), form=token.text,
                                   parent_id=str(parent_id), relation=relation)
            res.append(c)
        return res

Downloaded pre-trained UDPipe model for 'ru' language


# Parsing

In [ ]:
parser = UDPipeSpacyParser()

parsing_res = []
for i, sent in enumerate(sents):
  if i % 100 == 0:
      print(i)
  token_list = parser.parse(sent.text)
  cur_res = Sentence()
  cur_res.set_sent_id(sent.sent_id)
  cur_res.set_text(sent.text)
  for t in token_list:
    cur_res.add_token(t)
  parsing_res.append(cur_res)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700


In [ ]:
[parsing_res[0].tokens[0].__dict__[f] == sents[0].tokens[0].__dict__[f] for f in ['id', 'form', 'parent_id', 'relation']]

[True, True, True, True]

In [ ]:
import pickle
with open('udpipe_parsing.pickle', 'wb') as f:
     pickle.dump(parsing_res, f)

In [ ]:
from google.colab import files

files.download("udpipe_parsing.pickle")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>